In [10]:
import os
import requests
import pyconll
import json
from sklearn import metrics
from collections import Counter
from tqdm import tqdm

In [11]:
def askLLM(prompt, temperature):
    api_url = "http://127.0.0.1:8042/llama/"
    todo = {
        "prompts":[prompt], 
        "max_gen_len": 500, 
        "temperature" : temperature, 
        #"top_p" : top_p
    }
    response = requests.post(api_url, json=todo)
    resp = response.json()
    predicted_text = resp['responses'][0]['generation']
    endOfAnswer = predicted_text.find('\n')
    return predicted_text[:endOfAnswer]

In [12]:
# reading first 1020 dataset sentences
frases = pyconll.load_from_file('data/porttinari-base/Porttinari-base_test.conllu')
reviews = []
for sent in frases[:1020]:
    lista_ud=[]
    for token in sent:
        lista_ud.append([token.form, token.xpos, token.upos, token.deprel])
    reviews.append(lista_ud)

In [13]:
head = "Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) conforme os exemplos abaixo:\n\n"
promptStart = head
for review in reviews[:10]:
    entrada = "Entrada: "
    saida  = "Saida: "
    for token in review:
        if not token[2]:
            token[2] = 'None'
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    promptStart += entrada + "\n" + saida + "\n\n"
print(promptStart)

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) conforme os exemplos abaixo:

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são e

In [14]:
len(promptStart)

3084

## Search for optimal temperature

In [15]:
punctList = ['"', ',', '.', '[', ']', '?','!']
for temp in range(0,10):
    if temp:
        temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for revID, review in enumerate(tqdm(reviews[20:40])):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "
        fileName = f'resultados/llama/llama7B_t{temp}_{revID}.json'
        if os.path.isfile(fileName):
            with open(fileName) as file:
                tagsPred = json.load(file)
        else:            
            tries = 0
            while True:
                if tries > 10:
                    errors += 1
                    totalTries.append(tries)
                    with open(f'resultados/llama/llama7B_test_erro_t{temp}_{revID}.log', 'w') as f:
                        f.write(prompt)
                        f.write('======')
                        f.write(answer+'\n\n')
                        f.write('gold'+ str(goldSent) + str(len(goldSent))+'\n')
                        f.write(str(goldTokens)+'\n')
                        f.write('pred' + str(tagsSent) + str(len(tagsSent))+'\n')
                        f.write(str(tokens)+'\n')
                        f.write('\n\n')
                    tagsPred = ['None'] * len(goldSent)
                    break
                answer = askLLM(prompt, temperature=temp)
                tries += 1
                try:
                    tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                    tokensSent = []
                    tagsSent = []
                    for token in tokens:
                        if '/' in token:
                            tagsSent.append(token.split('/')[1])
                            tokensSent.append(token.split('/')[0])
                        else:
                            tagsSent.append('None')
                            tokensSent.append(token)
                except Exception as error:
                    continue
                idx = 0
                erroIdx = 0
                tagsPred = []
                for token, tag in zip(goldTokens, goldSent):
                    if idx >= len(tokensSent):
                        tagsPred.append('None')
                        erroIdx += 1
                        continue           
                    if tokensSent[idx].lower() == token.lower():
                        tagsPred.append(tagsSent[idx])
                    else:
                        if idx == 0:
                            while (tokensSent[idx].lower() != token.lower() and idx < 5):
                                idx += 1
                                if idx >= len(tokensSent):
                                    idx -= 1
                                    break
                        if tokensSent[idx].lower() == token.lower():
                            tagsPred.append(tagsSent[idx])
                        else:
                            tagsPred.append('None')
                            erroIdx += 1
                            continue
                    idx += 1
                if len(tagsPred) == len(goldSent) and erroIdx < 10:
                    totalTries.append(tries)
                    break
        with open(fileName, 'w') as f:
            json.dump(tagsPred, f)
        pred += tagsPred
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/llama7B_test_t{temp}.json', 'w') as f:
        json.dump(pred, f)
    #with open(f'resultados/gold.json', 'w') as f:
    #    json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0


100%|█████████████████████████████████████████████████████████████████| 20/20 [10:09<00:00, 30.48s/it]
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `z

gold 315 pred 315 errors 3
              precision    recall  f1-score   support

         ADJ       0.72      0.72      0.72        18
         ADP       0.78      0.62      0.69        45
         ADV       0.70      0.54      0.61        13
         AUX       0.67      0.50      0.57        16
       CCONJ       1.00      0.75      0.86         8
         DET       0.73      0.74      0.74        47
        INTJ       0.00      0.00      0.00         1
        NOUN       0.98      0.66      0.79        62
         NUM       0.50      0.60      0.55         5
        None       0.12      0.33      0.18        21
        PRON       0.60      0.38      0.46        16
       PROPN       0.70      0.86      0.78        22
       SCONJ       1.00      0.40      0.57         5
         SYM       0.00      0.00      0.00         1
        VERB       0.70      0.80      0.75        35

    accuracy                           0.64       315
   macro avg       0.61      0.53      0.55       315

100%|█████████████████████████████████████████████████████████████████| 20/20 [10:17<00:00, 30.89s/it]
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `z

gold 315 pred 315 errors 2
              precision    recall  f1-score   support

         ADJ       0.81      0.72      0.76        18
         ADP       0.75      0.60      0.67        45
         ADV       0.58      0.54      0.56        13
         AUX       0.69      0.56      0.62        16
       CCONJ       1.00      0.88      0.93         8
         DET       0.69      0.77      0.73        47
        INTJ       0.00      0.00      0.00         1
        NOUN       0.93      0.68      0.79        62
         NUM       0.50      0.80      0.62         5
        None       0.14      0.29      0.19        21
        PRON       0.55      0.38      0.44        16
       PROPN       0.66      0.86      0.75        22
       SCONJ       1.00      0.40      0.57         5
         SYM       1.00      1.00      1.00         1
        VERB       0.78      0.89      0.83        35

    accuracy                           0.67       315
   macro avg       0.67      0.62      0.63       315

...

100%|█████████████████████████████████████████████████████████████████| 20/20 [06:52<00:00, 20.63s/it]
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

gold 315 pred 315 errors 0
              precision    recall  f1-score   support

         ADJ       0.53      0.50      0.51        18
         ADP       0.78      0.69      0.73        45
         ADV       0.33      0.23      0.27        13
         AND       0.00      0.00      0.00         0
         AUX       0.44      0.44      0.44        16
       CCONJ       1.00      0.75      0.86         8
         DET       0.70      0.74      0.72        47
        INTJ       0.00      0.00      0.00         1
        NOUN       0.94      0.55      0.69        62
         NUM       0.31      1.00      0.48         5
        None       0.22      0.33      0.26        21
        PRON       0.57      0.50      0.53        16
       PROPN       0.70      0.95      0.81        22
       SCONJ       0.80      0.80      0.80         5
         SYM       1.00      1.00      1.00         1
        VERB       0.69      0.83      0.75        35

    accuracy                           0.63       315

100%|█████████████████████████████████████████████████████████████████| 20/20 [05:47<00:00, 17.36s/it]

gold 315 pred 315 errors 0
              precision    recall  f1-score   support

         ADJ       0.60      0.33      0.43        18
         ADP       0.84      0.60      0.70        45
         ADV       0.21      0.23      0.22        13
         AND       0.00      0.00      0.00         0
         AUX       0.61      0.69      0.65        16
       CCONJ       1.00      0.50      0.67         8
        CONJ       0.00      0.00      0.00         0
         DET       0.72      0.62      0.67        47
        INTJ       0.00      0.00      0.00         1
        NOUN       0.89      0.55      0.68        62
         NUM       0.29      0.80      0.42         5
        None       0.16      0.33      0.22        21
        PRON       0.44      0.44      0.44        16
       PROPN       0.60      0.95      0.74        22
       SCONJ       0.00      0.00      0.00         5
         SYM       0.00      0.00      0.00         1
        VERB       0.78      0.89      0.83        35



/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home

## 0.2 full experiment

In [16]:
punctList = ['"', ',', '.', '[', ']', '?','!']
for temp in range(2,3):
    if temp:
        temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for revID, review in enumerate(tqdm(reviews[20:])):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "
        fileName = f'resultados/llama/llama7B_t{temp}_{revID}.json'
        if os.path.isfile(fileName):
            with open(fileName) as file:
                tagsPred = json.load(file)
        else:            
            tries = 0
            while True:
                if tries > 10:
                    errors += 1
                    totalTries.append(tries)
                    with open(f'resultados/llama/llama7B_erro_t{temp}_{revID}.log', 'w') as f:
                        f.write(prompt)
                        f.write('======')
                        f.write(answer+'\n\n')
                        f.write('gold'+ str(goldSent) + str(len(goldSent))+'\n')
                        f.write(str(goldTokens)+'\n')
                        f.write('pred' + str(tagsSent) + str(len(tagsSent))+'\n')
                        f.write(str(tokens)+'\n')
                        f.write('\n\n')
                    tagsPred = ['None'] * len(goldSent)
                    break
                answer = askLLM(prompt, temperature=temp)
                tries += 1
                try:
                    tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                    tokensSent = []
                    tagsSent = []
                    for token in tokens:
                        if '/' in token:
                            tagsSent.append(token.split('/')[1])
                            tokensSent.append(token.split('/')[0])
                        else:
                            tagsSent.append('None')
                            tokensSent.append(token)
                except Exception as error:
                    continue
                idx = 0
                erroIdx = 0
                tagsPred = []
                for token, tag in zip(goldTokens, goldSent):
                    if idx >= len(tokensSent):
                        tagsPred.append('None')
                        erroIdx += 1
                        continue           
                    if tokensSent[idx].lower() == token.lower():
                        tagsPred.append(tagsSent[idx])
                    else:
                        if idx == 0:
                            while (tokensSent[idx].lower() != token.lower() and idx < 5):
                                idx += 1
                                if idx >= len(tokensSent):
                                    idx -= 1
                                    break
                        if tokensSent[idx].lower() == token.lower():
                            tagsPred.append(tagsSent[idx])
                        else:
                            tagsPred.append('None')
                            erroIdx += 1
                            continue
                    idx += 1
                if len(tagsPred) == len(goldSent) and erroIdx < 10:
                    totalTries.append(tries)
                    break
        with open(fileName, 'w') as f:
            json.dump(tagsPred, f)
        pred += tagsPred
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/llama7B_t{temp}.json', 'w') as f:
        json.dump(pred, f)
    #with open(f'resultados/gold.json', 'w') as f:
    #    json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0.2


100%|███████████████████████████████████████████████████████████| 1000/1000 [6:28:13<00:00, 23.29s/it]

gold 18690 pred 18690 errors 55
              precision    recall  f1-score   support

         ADJ       0.63      0.61      0.62       996
         ADP       0.82      0.73      0.77      2943
         ADV       0.63      0.68      0.65       759
         AUX       0.58      0.63      0.60       592
       CCONJ       1.00      0.70      0.82       497
        CONJ       0.00      0.00      0.00         0
         DET       0.74      0.81      0.77      2880
        INTJ       0.00      0.00      0.00         3
        NOUN       0.92      0.69      0.79      3757
         NUM       0.58      0.70      0.63       364
        None       0.20      0.45      0.28      1208
        PRON       0.59      0.39      0.47       771
       PROPN       0.73      0.78      0.75      1290
       PUNCT       0.82      0.40      0.53       222
       SCONJ       0.49      0.28      0.35       277
        SQUO       0.00      0.00      0.00         0
         SYM       0.98      0.70      0.82      


/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home

## Other tests

In [13]:
gold = []
pred = []
totalTries = []
errors = 0
for review in tqdm(reviews[20:]):
    prompt = promptStart
    entrada = "Entrada: "
    saida  = "Saida: "
    goldSent = []
    for token in review:
        if not token[2]:
            token[2] = 'None'
        gold.append(token[2])
        goldSent.append(token[2])
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    prompt += entrada + "\nSaída: "
    tries = 0
    while True:
        if tries > 15:
            errors += 1
            tagsSent = ['None'] * len(goldSent)
            totalTries.append(tries)
            break
        answer = askLamma(prompt)
        tries += 1
        #print(answer)
        try:
            tagsSent = [token.split('/')[1] for token in answer.split(' ')]
        except:
            #print('retry except', tries)
            continue
        if len(tagsSent) == len(goldSent):
            #print('retry', tries)
            totalTries.append(tries)
            break
    pred += tagsSent
    #print('=======')
print('gold',len(gold),'pred',len(pred),'errors',errors)
print(metrics.classification_report(gold, pred))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [22:23:29<00:00, 80.61s/it]

gold 21111 pred 21111 errors 281
              precision    recall  f1-score   support

                   0.00      0.00      0.00         0
      .PUNCT       0.00      0.00      0.00         0
         ADJ       0.55      0.44      0.49       996
         ADP       0.84      0.59      0.69      2943
         ADV       0.57      0.43      0.49       759
         AUX       0.53      0.47      0.50       592
       CCONJ       0.97      0.49      0.65       497
        CONJ       0.00      0.00      0.00         0
         DET       0.73      0.62      0.67      2880
        INTJ       0.00      0.00      0.00         3
        NOUN       0.88      0.50      0.64      3757
         NUM       0.56      0.55      0.55       364
        None       0.11      0.66      0.19      1208
          OR       0.00      0.00      0.00         0
        PRON       0.44      0.27      0.33       771
       PROPN       0.73      0.54      0.62      1290
       PUNCT       0.97      0.56      0.70     


/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home

In [15]:
print(totalTries)

[2, 1, 16, 1, 2, 2, 16, 2, 1, 16, 1, 16, 16, 16, 1, 1, 15, 2, 5, 16, 1, 2, 1, 2, 1, 16, 1, 4, 2, 1, 1, 16, 6, 1, 4, 16, 16, 4, 3, 16, 1, 1, 1, 16, 16, 8, 1, 16, 1, 2, 2, 4, 2, 9, 1, 11, 1, 1, 1, 3, 1, 1, 7, 16, 16, 5, 2, 2, 16, 1, 1, 1, 1, 2, 1, 1, 16, 16, 16, 1, 2, 16, 16, 2, 10, 16, 16, 1, 6, 1, 1, 16, 1, 5, 16, 1, 16, 4, 1, 4, 8, 1, 3, 16, 1, 1, 16, 1, 7, 1, 1, 2, 6, 16, 16, 5, 1, 1, 12, 1, 11, 1, 9, 16, 5, 1, 1, 16, 2, 16, 16, 2, 10, 16, 3, 16, 1, 3, 1, 1, 8, 7, 16, 3, 16, 1, 16, 16, 16, 5, 9, 5, 5, 2, 10, 1, 4, 16, 1, 1, 2, 16, 11, 5, 2, 6, 1, 1, 16, 1, 8, 4, 1, 1, 16, 2, 1, 1, 2, 1, 2, 3, 7, 1, 16, 2, 1, 1, 16, 1, 1, 13, 16, 16, 16, 7, 4, 16, 1, 10, 1, 16, 16, 6, 16, 1, 2, 1, 1, 1, 4, 3, 16, 4, 16, 1, 12, 16, 15, 1, 16, 4, 7, 1, 1, 2, 16, 2, 1, 1, 2, 16, 2, 16, 9, 2, 1, 1, 1, 16, 2, 1, 16, 2, 7, 16, 16, 3, 1, 11, 16, 16, 16, 1, 4, 1, 1, 6, 16, 6, 2, 1, 16, 16, 1, 16, 16, 1, 3, 1, 4, 16, 10, 10, 10, 8, 1, 16, 6, 16, 1, 7, 16, 1, 4, 10, 1, 16, 1, 1, 2, 1, 4, 1, 1, 1, 6, 5, 1, 5, 1,

In [30]:
for temp in range(1,10):
    temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for review in tqdm(reviews[20:30]):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "
        goldSent = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            gold.append(token[2])
            goldSent.append(token[2])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "
        tries = 0
        while True:
            if tries > 10:
                errors += 1
                tagsSent = ['None'] * len(goldSent)
                totalTries.append(tries)
                print(prompt)
                print('======')
                print(answer,'\n\n')
                break
            answer = askLamma(prompt, temperature=0.1, top_p=0.1)
            tries += 1
            #print(answer)
            try:
                tagsSent = [token.split('/')[1] for token in answer.split(' ')]
            except:
                #print('retry except', tries)
                continue
            if len(tagsSent) == len(goldSent):
                #print('retry', tries)
                totalTries.append(tries)
                break
        pred += tagsSent
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0.1


 30%|███████████████████████████████████████████████████▉                                                                                                                         | 3/10 [02:33<07:39, 65.58s/it]

Atuando como linguista e preservando a acentuação original, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) como nos exemplos abaixo.

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por 

 70%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                    | 7/10 [05:18<02:59, 59.78s/it]

Atuando como linguista e preservando a acentuação original, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) como nos exemplos abaixo.

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por 

...

Atuando como linguista e preservando a acentuação original, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) como nos exemplos abaixo.

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por 

 70%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                    | 7/10 [05:19<02:59, 59.87s/it]

Atuando como linguista e preservando a acentuação original, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) como nos exemplos abaixo.

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [07:54<00:00, 47.41s/it]

Atuando como linguista e preservando a acentuação original, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) como nos exemplos abaixo.

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por 

In [34]:
head = "Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):\n\n"
promptStart = head
for review in reviews[:10]:
    entrada = "Entrada: "
    saida  = "Saida: "
    for token in review:
        if not token[2]:
            token[2] = 'None'
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    promptStart += entrada + "\n" + saida + "\n\n"
print(promptStart)

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

In [87]:
punctList = ['"', ',', '.', '[', ']', '?','!']
             #, '"/PUNCT', ',/PUNCT', './PUNCT']
#punctList = []
#for temp in range(1,10):
    #temp = temp/10
temp = 0.7
print('temp',temp)
gold = []
pred = []
totalTries = []
errors = 0
for review in tqdm(reviews[20:]):
    prompt = promptStart
    entrada = "Entrada: "
    saida  = "Saida: "
    goldSent = []
    goldTokens = []
    for token in review:
        if not token[2]:
            token[2] = 'None'
        #print(token)
        if token[0] not in punctList:
            gold.append(token[2])
            goldSent.append(token[2])
            goldTokens.append(token[0])
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    prompt += entrada + "\nSaída: "
    #print(prompt)
    #sdad
    tries = 0
    while True:
        if tries > 10:
            errors += 1
            
            totalTries.append(tries)
            print(prompt)
            print('======')
            print(answer,'\n\n')
            print('gold', goldSent, len(goldSent))
            print(goldTokens)
            print('pred', tagsSent, len(tagsSent))
            print(tokens)
            print('\n\n')
            tagsPred = ['None'] * len(goldSent)
            #dsdaasd
            break
        answer, resp = askLamma(prompt, temperature=0.1, top_p=0.1)
        tries += 1
        #print(answer)
        try:
            tokens = [token for token in answer.split(' ') if token[0] not in punctList]
            tokensSent = []
            tagsSent = []
            for token in tokens:
                if '/' in token:
                    tagsSent.append(token.split('/')[1])
                    tokensSent.append(token.split('/')[0])
                else:
                    tagsSent.append('None')
                    tokensSent.append(token)
            #tagsSent = [token.split('/')[1] for token in answer.split(' ') if token[0] not in punctList]
        #except:
        except Exception as error:
            #print('retry except', tries, error)
            #print('prompt',prompt)
            #print('ans:',answer)
            #print('resp:',resp)
            continue

        #print('gt:',len(goldTokens),goldTokens)
        #print('gs:',len(goldSent),goldSent)
        #print('pt:',len(tokensSent),tokensSent)
        #print('ts:',len(tagsSent),tagsSent)

        idx = 0
        erroIdx = 0
        tagsPred = []
        for token, tag in zip(goldTokens, goldSent):
            #print(token,tag)
            #print(tokensSent)
            #print(idx)
            if idx >= len(tokensSent):
                tagsPred.append('None')
                erroIdx += 1
                continue           
            if tokensSent[idx].lower() == token.lower():
                #print('ok')
                tagsPred.append(tagsSent[idx])
            else:
                if idx == 0:
                    #print(token, tokensSent)
                    while (tokensSent[idx].lower() != token.lower() and idx < 5):
                        idx += 1
                        if idx >= len(tokensSent):
                            idx -= 1
                            break
                if tokensSent[idx].lower() == token.lower():
                    #print('ok')
                    tagsPred.append(tagsSent[idx])
                else:
                    #print('not ok')
                    tagsPred.append('None')
                    erroIdx += 1
                    continue
            idx += 1
        #print(erroIdx)
        #print('tp:',len(tagsPred),tagsPred)
        #asda
        #for 
        
        if len(tagsPred) == len(goldSent) and erroIdx < 10:
            #print('retry', tries)
            totalTries.append(tries)
            break
    pred += tagsPred
    #print('=======')
print('gold',len(gold),'pred',len(pred),'errors',errors)
print(metrics.classification_report(gold, pred))
print("\n\n\n\n######\n\n\n")

temp 0.7


  3%|█████▎                                                                                                                                                                 | 32/1000 [10:49<11:13:07, 41.72s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

  8%|█████████████                                                                                                                                                          | 78/1000 [25:21<11:37:39, 45.40s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

...

 94%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎          | 936/1000 [4:41:08<51:57, 48.71s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [5:00:09<00:00, 18.01s/it]

gold 18690 pred 18690 errors 7
              precision    recall  f1-score   support

         ADJ       0.59      0.62      0.60       996
         ADP       0.82      0.73      0.77      2943
         ADV       0.59      0.62      0.60       759
         AND       0.00      0.00      0.00         0
         AUX       0.54      0.60      0.57       592
       CCONJ       0.99      0.66      0.79       497
        CONJ       0.00      0.00      0.00         0
         DET       0.71      0.82      0.76      2880
         HAD       0.00      0.00      0.00         0
        INTJ       0.00      0.00      0.00         3
    NEGATIVE       0.00      0.00      0.00         0
        NOUN       0.92      0.71      0.80      3757
         NUM       0.58      0.71      0.64       364
      NUMBER       0.00      0.00      0.00         0
        None       0.24      0.40      0.30      1208
          OR       0.00      0.00      0.00         0
         PLU       0.00      0.00      0.00       


/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home

## Testing temperature from 0.6 to 0.9

In [97]:
punctList = ['"', ',', '.', '[', ']', '?','!']
             #, '"/PUNCT', ',/PUNCT', './PUNCT']
#punctList = []
for temp in range(6,10):
    temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for review in tqdm(reviews[20:]):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            #print(token)
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "
        #print(prompt)
        #sdad
        tries = 0
        while True:
            if tries > 10:
                errors += 1
                
                totalTries.append(tries)
                print(prompt)
                print('======')
                print(answer,'\n\n')
                print('gold', goldSent, len(goldSent))
                print(goldTokens)
                print('pred', tagsSent, len(tagsSent))
                print(tokens)
                print('\n\n')
                tagsPred = ['None'] * len(goldSent)
                #dsdaasd
                break
            answer, resp = askLamma(prompt, temperature=temp)
            tries += 1
            #print(answer)
            try:
                tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                tokensSent = []
                tagsSent = []
                for token in tokens:
                    if '/' in token:
                        tagsSent.append(token.split('/')[1])
                        tokensSent.append(token.split('/')[0])
                    else:
                        tagsSent.append('None')
                        tokensSent.append(token)
                #tagsSent = [token.split('/')[1] for token in answer.split(' ') if token[0] not in punctList]
            #except:
            except Exception as error:
                #print('retry except', tries, error)
                #print('prompt',prompt)
                #print('ans:',answer)
                #print('resp:',resp)
                continue
    
            #print('gt:',len(goldTokens),goldTokens)
            #print('gs:',len(goldSent),goldSent)
            #print('pt:',len(tokensSent),tokensSent)
            #print('ts:',len(tagsSent),tagsSent)
    
            idx = 0
            erroIdx = 0
            tagsPred = []
            for token, tag in zip(goldTokens, goldSent):
                #print(token,tag)
                #print(tokensSent)
                #print(idx)
                if idx >= len(tokensSent):
                    tagsPred.append('None')
                    erroIdx += 1
                    continue           
                if tokensSent[idx].lower() == token.lower():
                    #print('ok')
                    tagsPred.append(tagsSent[idx])
                else:
                    if idx == 0:
                        #print(token, tokensSent)
                        while (tokensSent[idx].lower() != token.lower() and idx < 5):
                            idx += 1
                            if idx >= len(tokensSent):
                                idx -= 1
                                break
                    if tokensSent[idx].lower() == token.lower():
                        #print('ok')
                        tagsPred.append(tagsSent[idx])
                    else:
                        #print('not ok')
                        tagsPred.append('None')
                        erroIdx += 1
                        continue
                idx += 1
            #print(erroIdx)
            #print('tp:',len(tagsPred),tagsPred)
            #asda
            #for 
            
            if len(tagsPred) == len(goldSent) and erroIdx < 10:
                #print('retry', tries)
                totalTries.append(tries)
                break
        pred += tagsPred
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/llama7btemp{temp}.json', 'w') as f:
        json.dump(pred, f)
    with open(f'resultados/gold.json', 'w') as f:
        json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0.6


  8%|█████████████                                                                                                                                                          | 78/1000 [23:45<12:38:16, 49.34s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

 19%|████████████████████████████████                                                                                                                                      | 193/1000 [59:51<11:41:39, 52.17s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 986/1000 [4:59:46<11:39, 49.96s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [5:04:59<00:00, 18.30s/it]

gold 18690 pred 18690 errors 5
              precision    recall  f1-score   support

         ADJ       0.50      0.54      0.52       996
         ADP       0.84      0.66      0.74      2943
         ADV       0.46      0.52      0.49       759
       AFTER       0.00      0.00      0.00         0
         AND       0.00      0.00      0.00         0
       ATDIC       0.00      0.00      0.00         0
         AUX       0.47      0.58      0.52       592
          Au       0.00      0.00      0.00         0
       CCONJ       0.95      0.61      0.74       497
         COM       0.00      0.00      0.00         0
        CONJ       0.00      0.00      0.00         0
        DATE       0.00      0.00      0.00         0
         DEC       0.00      0.00      0.00         0
         DET       0.68      0.76      0.72      2880
          IF       0.00      0.00      0.00         0
         INC       0.00      0.00      0.00         0
         IND       0.00      0.00      0.00       


/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home

## From 0.1 to 0.5

In [98]:
punctList = ['"', ',', '.', '[', ']', '?','!']
             #, '"/PUNCT', ',/PUNCT', './PUNCT']
#punctList = []
for temp in range(1,6):
    temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for review in tqdm(reviews[20:]):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            #print(token)
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "
        #print(prompt)
        #sdad
        tries = 0
        while True:
            if tries > 10:
                errors += 1
                
                totalTries.append(tries)
                print(prompt)
                print('======')
                print(answer,'\n\n')
                print('gold', goldSent, len(goldSent))
                print(goldTokens)
                print('pred', tagsSent, len(tagsSent))
                print(tokens)
                print('\n\n')
                tagsPred = ['None'] * len(goldSent)
                #dsdaasd
                break
            answer, resp = askLamma(prompt, temperature=temp)
            tries += 1
            #print(answer)
            try:
                tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                tokensSent = []
                tagsSent = []
                for token in tokens:
                    if '/' in token:
                        tagsSent.append(token.split('/')[1])
                        tokensSent.append(token.split('/')[0])
                    else:
                        tagsSent.append('None')
                        tokensSent.append(token)
                #tagsSent = [token.split('/')[1] for token in answer.split(' ') if token[0] not in punctList]
            #except:
            except Exception as error:
                #print('retry except', tries, error)
                #print('prompt',prompt)
                #print('ans:',answer)
                #print('resp:',resp)
                continue
    
            #print('gt:',len(goldTokens),goldTokens)
            #print('gs:',len(goldSent),goldSent)
            #print('pt:',len(tokensSent),tokensSent)
            #print('ts:',len(tagsSent),tagsSent)
    
            idx = 0
            erroIdx = 0
            tagsPred = []
            for token, tag in zip(goldTokens, goldSent):
                #print(token,tag)
                #print(tokensSent)
                #print(idx)
                if idx >= len(tokensSent):
                    tagsPred.append('None')
                    erroIdx += 1
                    continue           
                if tokensSent[idx].lower() == token.lower():
                    #print('ok')
                    tagsPred.append(tagsSent[idx])
                else:
                    if idx == 0:
                        #print(token, tokensSent)
                        while (tokensSent[idx].lower() != token.lower() and idx < 5):
                            idx += 1
                            if idx >= len(tokensSent):
                                idx -= 1
                                break
                    if tokensSent[idx].lower() == token.lower():
                        #print('ok')
                        tagsPred.append(tagsSent[idx])
                    else:
                        #print('not ok')
                        tagsPred.append('None')
                        erroIdx += 1
                        continue
                idx += 1
            #print(erroIdx)
            #print('tp:',len(tagsPred),tagsPred)
            #asda
            #for 
            
            if len(tagsPred) == len(goldSent) and erroIdx < 10:
                #print('retry', tries)
                totalTries.append(tries)
                break
        pred += tagsPred
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/llama7btemp{temp}.json', 'w') as f:
        json.dump(pred, f)
    with open(f'resultados/gold.json', 'w') as f:
        json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0.1


  2%|██▌                                                                                                                                                                    | 15/1000 [04:57<13:04:08, 47.77s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

  2%|███                                                                                                                                                                    | 18/1000 [07:32<16:22:48, 60.05s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 986/1000 [5:20:58<14:35, 62.52s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [5:24:17<00:00, 19.46s/it]

gold 18690 pred 18690 errors 14
              precision    recall  f1-score   support

         ADJ       0.60      0.63      0.62       996
         ADP       0.82      0.73      0.77      2943
         ADV       0.60      0.66      0.63       759
         AUX       0.56      0.62      0.59       592
       CCONJ       0.98      0.68      0.81       497
        CONJ       0.00      0.00      0.00         0
         DET       0.72      0.82      0.77      2880
        INTJ       0.00      0.00      0.00         3
        NOUN       0.92      0.70      0.79      3757
         NUM       0.58      0.69      0.63       364
        None       0.24      0.41      0.30      1208
        PRON       0.51      0.39      0.44       771
       PROPN       0.72      0.80      0.76      1290
       PUNCT       0.83      0.41      0.55       222
       SCONJ       0.42      0.28      0.33       277
      SQUOTE       0.00      0.00      0.00         0
         SYM       0.92      0.59      0.72      


/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home

## temp 0 experiment

In [100]:
punctList = ['"', ',', '.', '[', ']', '?','!']
             #, '"/PUNCT', ',/PUNCT', './PUNCT']
#punctList = []
for temp in range(0,1):
    #temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for review in tqdm(reviews[20:]):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            #print(token)
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "
        #print(prompt)
        #sdad
        tries = 0
        while True:
            if tries > 10:
                errors += 1
                
                totalTries.append(tries)
                print(prompt)
                print('======')
                print(answer,'\n\n')
                print('gold', goldSent, len(goldSent))
                print(goldTokens)
                print('pred', tagsSent, len(tagsSent))
                print(tokens)
                print('\n\n')
                tagsPred = ['None'] * len(goldSent)
                #dsdaasd
                break
            answer, resp = askLamma(prompt, temperature=temp)
            tries += 1
            #print(answer)
            try:
                tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                tokensSent = []
                tagsSent = []
                for token in tokens:
                    if '/' in token:
                        tagsSent.append(token.split('/')[1])
                        tokensSent.append(token.split('/')[0])
                    else:
                        tagsSent.append('None')
                        tokensSent.append(token)
                #tagsSent = [token.split('/')[1] for token in answer.split(' ') if token[0] not in punctList]
            #except:
            except Exception as error:
                #print('retry except', tries, error)
                #print('prompt',prompt)
                #print('ans:',answer)
                #print('resp:',resp)
                continue
    
            #print('gt:',len(goldTokens),goldTokens)
            #print('gs:',len(goldSent),goldSent)
            #print('pt:',len(tokensSent),tokensSent)
            #print('ts:',len(tagsSent),tagsSent)
    
            idx = 0
            erroIdx = 0
            tagsPred = []
            for token, tag in zip(goldTokens, goldSent):
                #print(token,tag)
                #print(tokensSent)
                #print(idx)
                if idx >= len(tokensSent):
                    tagsPred.append('None')
                    erroIdx += 1
                    continue           
                if tokensSent[idx].lower() == token.lower():
                    #print('ok')
                    tagsPred.append(tagsSent[idx])
                else:
                    if idx == 0:
                        #print(token, tokensSent)
                        while (tokensSent[idx].lower() != token.lower() and idx < 5):
                            idx += 1
                            if idx >= len(tokensSent):
                                idx -= 1
                                break
                    if tokensSent[idx].lower() == token.lower():
                        #print('ok')
                        tagsPred.append(tagsSent[idx])
                    else:
                        #print('not ok')
                        tagsPred.append('None')
                        erroIdx += 1
                        continue
                idx += 1
            #print(erroIdx)
            #print('tp:',len(tagsPred),tagsPred)
            #asda
            #for 
            
            if len(tagsPred) == len(goldSent) and erroIdx < 10:
                #print('retry', tries)
                totalTries.append(tries)
                break
        pred += tagsPred
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/llama7btemp{temp}.json', 'w') as f:
        json.dump(pred, f)
    with open(f'resultados/gold.json', 'w') as f:
        json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0


  2%|██▌                                                                                                                                                                    | 15/1000 [04:51<12:49:16, 46.86s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

  2%|███                                                                                                                                                                    | 18/1000 [07:25<16:13:52, 59.50s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 998/1000 [8:03:33<02:15, 67.87s/it]

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são eliminados por R$ 150 .
Saida

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [8:03:58<00:00, 29.04s/it]

gold 18690 pred 18690 errors 138
              precision    recall  f1-score   support

         ADJ       0.64      0.57      0.60       996
         ADP       0.82      0.66      0.73      2943
         ADV       0.62      0.61      0.62       759
         AUX       0.60      0.56      0.58       592
       CCONJ       0.99      0.62      0.76       497
        CONJ       0.00      0.00      0.00         0
         DET       0.74      0.75      0.74      2880
        INTJ       0.00      0.00      0.00         3
        NOUN       0.92      0.64      0.75      3757
         NUM       0.60      0.61      0.61       364
        None       0.14      0.47      0.21      1208
        PRON       0.60      0.36      0.45       771
       PROPN       0.74      0.72      0.73      1290
       PUNCT       0.86      0.35      0.50       222
       SCONJ       0.47      0.21      0.29       277
         SYM       1.00      0.53      0.69        74
        VERB       0.83      0.74      0.78     


/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home